# Code to Generate Biochar Atomistic Models

<img src="biochar.png" width="450" height="450">


## Open Pymol 

In [1]:
# Import pymol
from ipymol import viewer as pymol

In [2]:
# Initiate PyMol RPC module
import glob,os,sys,subprocess,random, math
from pymol_rpc import pymol_jupyter_builder
import numpy as np
from scipy.optimize import minimize
from sympy import symbols, Eq, solve

builder = pymol_jupyter_builder()
builder.start()

In [3]:
# Import cmd 
import xmlrpc.client as xmlrpclib
cmd = xmlrpclib.ServerProxy('http://localhost:9123')
cmd.reinitialize()

## Input: Biochar data

In [4]:
Biochar_Temp=float(input("Enter the temperature of pyrolysis in °C: "))

Enter the temperature of pyrolysis in °C:  500


In [5]:
# Hydroxyl groups, qualitative data from FTIR

if Biochar_Temp == 400:
    v = 0.03
elif Biochar_Temp == 500:
    v = 0.001
elif Biochar_Temp == 600 or Biochar_Temp == 700:
    v = 0
else:
    v = 0  # You may want to specify a default value in case Biochar_Temp doesn't match any of the above conditions

### Ultimate analysis

In [6]:
Carbon = float(input("Enter the value for Carbon: "))
Hydrogen = float(input("Enter the value for Hydrogen: "))
Oxygen = float(input("Enter the value for Oxygen: "))
Nitrogen = float(input("Enter the value for Nitrogen: "))
H_C = float(input("Enter the value for H/C molar ratio: "))
O_C = float(input("Enter the value for O/C molar ratio: "))
BridgheadCarbon=float(input("Enter the value for BridgheadCarbon: "))

Enter the value for Carbon:  0.8141
Enter the value for Hydrogen:  0.0350
Enter the value for Oxygen:  0.1498
Enter the value for Nitrogen:  0.0011
Enter the value for H/C molar ratio:  0.512
Enter the value for O/C molar ratio:  0.137
Enter the value for BridgheadCarbon:  0.488


### Multi-CP 13C NMR quantitaive data

In [7]:
Aromatic = float(input("Enter the value for Caro: "))
Carbonyl = float(input("Enter the value for Carbonyl: "))
Ester = float(input("Enter the value for Ester: "))          # Carboxyl/Lactone/Ester
Ether = float(input("Enter the value for Ether: "))
Aliphatic = float(input("Enter the value for Aliphatic: "))
Defect = float(input("Enter the value for Defect: "))        # Non-hexagonal rings
C_H_=float(input("Enter the value C-H: "))                   # If you don't have these values put zero
C_x_H_=float(input("Enter the value C-O-H: "))               # If you don't have these values put zero

Enter the value for Caro:  0.6733
Enter the value for Carbonyl:  0.0233
Enter the value for Ester:  0.0333
Enter the value for Ether:  0.1233
Enter the value for Aliphatic:  0.0333
Enter the value for Defect:  0.1133
Enter the value C-H:  0.31
Enter the value C-O-H:  0.47


### Aromatic cluster size distribution, obtained from BPCA.ipynb

In [8]:
system_size = float(input("System size selected on BPCA.ipynb: "))

System size selected on BPCA.ipynb:  10000


In [9]:
%store -r result
values=np.round(result[0]).astype(int)
print(values)

[ 0  3  0  0  0 10  0  0  0  0  0  0  2  0  0  0 11  3 62  3  0  0  7  3
  1  2  0  0  0  0  0  2  0]


## Step1: Include the aromatic clusters in PyMOL and create the grid

#### Note: All the molecules included in the BPCA_PAH should be in this folder as mol2 file 

In [10]:
cmd.load("fluorene.mol2", "fluorene")
cmd.load("phenalene.mol2", "phenalene")
cmd.load("phenanthrene.mol2", "phenanthrene")
cmd.load("anthracene.mol2", "anthracene")
cmd.load("tetracene.mol2", "tetracene")
cmd.load("pentacene.mol2", "pentacene")
cmd.load("pyrene.mol2", "pyrene")
cmd.load("chrysene.mol2", "chrysene")
cmd.load("benzo_a_fluorene.mol2", "benzo_a_fluorene")
cmd.load("benzo_b_fluoranthene.mol2", "benzo_b_fluoranthene")
cmd.load("benzo_b_fluorene.mol2", "benzo_b_fluorene")
cmd.load("coronene.mol2", "coronene")
cmd.load("perylene.mol2", "perylene")
cmd.load("benzo_a_pyrene.mol2", "benzo_a_pyrene")
cmd.load("benzo_g_h_i_perylene.mol2", "benzo_g_h_i_perylene")
cmd.load("circumpyrene.mol2", "circumpyrene")
cmd.load("circumcoronene.mol2", "circumcoronene")
cmd.load("circumovalene.mol2", "circumovalene")
cmd.load("pentatriacotaene.mol2", "pentatriacotaene")
cmd.load("circumcircumpyrene.mol2", "circumcircumpyrene")
cmd.load("c84.mol2", "c84")
cmd.load("N5.mol2", "m5")
cmd.load("N8.mol2", "N8")
cmd.load("N9.mol2", "N9")
cmd.load("N10.mol2", "N10")
cmd.load("N11.mol2", "N11")
cmd.load("N12.mol2", "N12")
cmd.load("N13.mol2", "N13")
cmd.load("N14.mol2", "N14")
cmd.load("N15.mol2", "N15")
cmd.load("N16.mol2", "N16")
cmd.load("N17.mol2", "N17")
cmd.load("N18.mol2", "N18")

1

In [11]:
cmd.alias("colour", "color cyan, (name c*); color red, (name o*); color white, (name h*); color blue, (name n*); color yellow, (name s); color brown, (name si)")

In [12]:
cmd.do('colour')
cmd.do('orient')
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')
cmd.bg_color("white")
cmd.set("ray_shadows", "off")

In [13]:
def generate_coordinates(num_coordinates):
    coordinates = []
    
    xmin, xmax = 0, 150
    ymin, ymax = -150, 0
    zmin, zmax = 0, 150

    num_per_dim = int(math.ceil(num_coordinates ** (1/3)))
    
    x_spacing = (xmax - xmin) / (num_per_dim - 1)
    y_spacing = (ymax - ymin) / (num_per_dim - 1)
    z_spacing = (zmax - zmin) / (num_per_dim - 1)
    
    count = 0
    for x in range(num_per_dim):
        for y in range(num_per_dim):
            for z in range(num_per_dim):
                if count >= num_coordinates:
                    break
                x_coord = xmin + x * x_spacing
                y_coord = ymin + y * y_spacing
                z_coord = zmin + z * z_spacing
                rotation_angles = [random.uniform(0, 360) for _ in range(3)]
                coordinates.append((x_coord, y_coord, z_coord, rotation_angles))
                count += 1
            if count >= num_coordinates:
                break
        if count >= num_coordinates:
            break

    return coordinates

coordinates = generate_coordinates(round(system_size*0.02))

In [14]:
object_list = cmd.get_object_list()
num_objects = len(object_list)
print("PAH types:", num_objects)

PAH types: 33


#### Insert all the PAH in PyMol according to the BPCA distribution

In [15]:
def copy_translate_delete(object_name, num_copies, coordinates, start_index):
    for i in range(1, num_copies + 1):
        cmd.copy(f"{object_name}{i}", object_name)
        cmd.translate(coordinates[start_index + i - 1], f"{object_name}{i}")
    cmd.delete(object_name)

for idx, object_name in enumerate(object_list):
    copy_translate_delete(object_name, values[idx], coordinates, num_objects)
    num_objects += values[idx]

cmd.reset()

object_list = cmd.get_object_list()
num_objects = len(object_list)
print("PAH clusters:", num_objects)

PAH clusters: 109


### Check the number of aromatic carbons

In [16]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_aromatic = atom_counts["C"]
h_aromatic = atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6155
Element H: 2146
Element N: 0
Element O: 0
Element S: 0
Element Cl: 0
Element Si: 0


## Step 2: Include cluster with non-hexagonal rings

#### Include a frequency for the Carbon Defects, to have the complete distribution of fragments.

In [17]:
# 1 Yes, 0 No
holes = float(input("Do you want to include holes within the structures?: "))  

Do you want to include holes within the structures?:  1


In [18]:
total_atoms=c_aromatic+h_aromatic
def calculate_factors(Biochar_Temp, total_atoms):
    if holes == 0:
        w = 0
        y = 1.1
        f = round(system_size*0.002)
    else:  
        # Adjustments based on the biochar production temperature
        if Biochar_Temp == 400:
            w = -0.25
            y = 1.5
            f = round(system_size*0.003)
        elif Biochar_Temp == 500:
            w = -0.04
            y = 1.25
            f = round(system_size*0.003)
        elif Biochar_Temp == 600:
            w = 0.005 
            y = 1.12 
            f = round(system_size*0.003)
        elif Biochar_Temp == 700:
            w = -0.05
            y = 1.1
            f = round(system_size*0.003)
            
    return w, y, f

w, y, f = calculate_factors(Biochar_Temp, total_atoms)
print("Adjusted factors - w:", w, "y:", y, "f:", f)

Adjusted factors - w: -0.04 y: 1.25 f: 30


In [19]:
def_list = [61,74, 93, 42, 45, 162, 67, 183, 192, 64, 72]
def_list_H = [23, 96, 126, 22, 32, 177, 83, 178, 175, 74, 28]
TotalC_Aromatic = c_aromatic
TotalH_Aromatic = h_aromatic

value =[]

def constraint(vector, def_list, def_list_H, TotalC_Aromatic, TotalH_Aromatic):
    
    # Total number of defective carbons
    value = [(def_list[i] * vector[i]) / (TotalC_Aromatic + (def_list[i] * vector[i])) for i in range(len(def_list))]
    constraint_value = sum(value)
     
    # Total H/C
    value_H_d = [def_list_H[i] * vector[i] for i in range(len(def_list))]
    value_C_d =  [def_list[i] * vector[i] for i in range(len(def_list))]      
    value_HC = (sum(value_H_d) + TotalH_Aromatic) / (TotalC_Aromatic + sum(value_C_d))
    
    # Total % of carbons
    value_C = (sum(value_C_d)  + TotalC_Aromatic) / (TotalC_Aromatic + TotalH_Aromatic + sum(value_H_d) + sum(value_C_d))
    constraint_value_C = value_C

    # Minimize errors
    error_HC = np.abs(value_HC - (H_C+w))/(H_C+w) # We need more hydrogen to create cross-links    
    error_Def = np.abs(constraint_value - Defect*y)/(Defect*y) # We need more carbon defect to create holes
    
    Total_error = error_Def + error_HC

    return Total_error*100

# Initial guess for the optimization
initial_vector = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

bounds = [(0, f) for i in range(len(def_list))]

# Optimize the objective function (set to zero) subject to constraints
result = minimize(lambda x: 0, initial_vector, method='slsqp', constraints={'type': 'eq', 'fun': constraint, 'args': (def_list, def_list_H, TotalC_Aromatic, TotalH_Aromatic)}, bounds=bounds, tol=1e-5)

# Extract the optimized vector
vector = result.x.round().astype(int)

# Print the results
print("Value:", constraint(vector, def_list, def_list_H, TotalC_Aromatic, TotalH_Aromatic))
print("Optimized Vector:", vector)

Value: 3.007945083773134
Optimized Vector: [0 4 5 0 0 0 2 0 0 0 0]


In [20]:
# New vector with defective structures included
values = np.append(values, vector).astype(int)
print(values)

[ 0  3  0  0  0 10  0  0  0  0  0  0  2  0  0  0 11  3 62  3  0  0  7  3
  1  2  0  0  0  0  0  2  0  0  4  5  0  0  0  2  0  0  0  0]


In [21]:
# Upload defects
cmd.load("defect1.mol2", "defect0")
cmd.load("defect2.mol2", "defect2")
cmd.load("defect3.mol2", "defect3")
cmd.load("defect4.mol2", "defect4")
cmd.load("defect5.mol2", "defect5")
cmd.load("defect6.mol2", "defect6")
cmd.load("defect7.mol2", "defect7")
cmd.load("defect8.mol2", "defect8")
cmd.load("defect9.mol2", "defect9")
cmd.load("defect10.mol2", "defect10")
cmd.load("defect11.mol2", "defect11")

1

In [22]:
# List of defect object names and their corresponding indices in vector `values`
defect_object_names = [
    ("defect0", 33),
    ("defect3", 35),
    ("defect2", 34),
    ("defect4", 36),
    ("defect5", 37),
    ("defect6", 38),
    ("defect7", 39),
    ("defect8", 40),
    ("defect9", 41),
    ("defect10", 42),
    ("defect11", 43)
]

defect_offset = num_objects

for object_name, value_index in defect_object_names:
    copy_translate_delete(object_name, values[value_index], coordinates, defect_offset)
    defect_offset += values[value_index]

cmd.reset()
object_list = cmd.get_object_list()
num_objects = len(object_list)
print("Number of objects:", num_objects)
cmd.reset()

Number of objects: 120


1

In [23]:
cmd.do('colour')
cmd.do('comp')
cmd.do('atomdata')
cmd.do('orient')
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')

### Organize the structures without overlapping them; this will allow you to visualize better the changes made

In [24]:
min_angle = 1
max_angle = 359.0

num_objects = len(object_list)

positions = []
used_positions = set()
angles = []
xmin, xmax = 0, round(system_size*0.015)
ymin, ymax = -round(system_size*0.015), 0
zmin, zmax = 0, round(system_size*0.015)

# Minimum distance between any two positions to avoid overlap
min_distance = round(system_size*0.005)
grid_divisions = int(math.ceil(num_objects ** (1 / 3)))

grid_size_x = min_distance
grid_size_y = min_distance
grid_size_z = min_distance

# Generate all possible grid positions
all_positions = [
    (xmin + i * grid_size_x, ymin + j * grid_size_y, zmin + k * grid_size_z)
    for i in range(grid_divisions)
    for j in range(grid_divisions)
    for k in range(grid_divisions)
]

random.shuffle(all_positions)
assert len(all_positions) >= num_objects, "Not enough positions for all objects."

# Assign positions to objects
for idx, name in enumerate(object_list):
    x, y, z = all_positions[idx]
    positions.append((x, y, z))

    rotation_angles = [random.uniform(min_angle, max_angle) for _ in range(3)]
    angles.append((rotation_angles, name))

    for axis, angle in zip(['x', 'y', 'z'], rotation_angles):
        rotation_command = "rotate {0}, {1}, {2}".format(axis, angle, name)
        cmd.do(rotation_command)

    # Translate the object to the new position
    cmd.translate((x, y, z), name)

cmd.refresh()

object_list = cmd.get_object_list()
num_objects = len(object_list)
print("Number of objects:", num_objects)

Number of objects: 120


In [25]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total= atom_counts["C"]
h_total= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7050
Element H: 3326
Element N: 0
Element O: 0
Element S: 0
Element Cl: 0
Element Si: 0


In [26]:
c_defect=c_total-c_aromatic
print("Defect carbon:", c_defect)

Defect carbon: 895


## Step 3: Include functional groups

### Add additional groups as needed

In [27]:
# Molecule type and functional groups to be included, information obtained from FTIR, CNMR, and XPS 
# The user defines these:             
Furan=0           # Change: interior C 
Pyrrolic=0        # Change: interior C
Fluorene=0        # Change: interior 
Thiophene=0       # Change: interior

# To reduce the error of not having enough information for oxygen, 
# it is necessary to sacrifice the precision of functional groups 
# that contain oxygen, such as ether, carbonyl, ester, and carboxylic groups.
if holes == 1:
    if Biochar_Temp == 400:
        z = -0.05   # A factor to fix oxygen based on ether, carbonyl, and ester groups
        u = 0       # How much of the aliphatic is methyl
        b = 0       # How much of the aliphatic is CH2CH3
        p = 1       # How much of the aliphatic is (CH2)-(CH2)-CH3
        h = 0.05    # A factor to fix oxygen based on aliphatic groups
        q = -0.01   # A factor to fix oxygen based on defects
    elif Biochar_Temp == 500:
        z = 0.05
        u = 1/3
        b = 2/3
        p = 0
        h = 0
        q = 0.05
    elif Biochar_Temp == 600:
        z = 0.05
        u = 0
        b = 0
        p = 0
        h = 0
        q = 0.06
    elif Biochar_Temp == 700:
        z = 0.1
        u = 0
        b = 0
        p = 0
        h = 0
        q = 0.06
else:
    if Biochar_Temp == 400:
        z = -0.05   # A factor to fix oxygen based on ether, carbonyl, and ester groups
        u = 0       # How much of the aliphatic is methyl
        b = 0       # How much of the aliphatic is CH2CH3
        p = 1       # How much of the aliphatic is (CH2)-(CH2)-CH3
        h = 0.05    # A factor to fix oxygen based on aliphatic groups
        q = 0       # A factor to fix oxygen based on defects
    elif Biochar_Temp == 500:
        z = 0.05
        u = 1/3
        b = 2/3
        p = 0
        h = 0
        q = 0
    elif Biochar_Temp == 600:
        z = 0.05
        u = 0
        b = 0
        p = 0
        h = 0
        q = 0
    elif Biochar_Temp == 700:
        z = 0.1
        u = 0
        b = 0
        p = 0
        h = 0
        q = 0

### Fix carbon, nitrogen, oxygen, and hydrogen

In [28]:
# Variables
c, carb, ester, eth, alip, defe, c_nonar,aro_ring, methyl, ali_chain, hydro, aro, nitro_groups, aniline, pyridin, quaternaryN, MW, oxy, hydroxyl,ali_chain_2 = symbols('c carb ester eth aro_ring c_nonar alip defe methyl ali_chain hydro aro nitro_groups aniline pyridin quaternaryN MW oxy hydroxyl ali_chain_2')
# Equations
equations = [
    
    # Carbon
    Eq(c_nonar, defe+methyl+ali_chain*2+ali_chain_2*3+carb+ester),# Non-aromatic carbons in the model
    Eq(defe,Defect*c*(1+q)),
    Eq(c,(c_nonar+aro_ring)),
    Eq(aro,(aro_ring-aniline-hydroxyl-ester-carb-methyl-ali_chain-eth)), # Actual number of aromatic carbons in the model
    Eq(aro_ring,(c_aromatic-eth-pyridin-quaternaryN)),            # Aromatic carbons removed
    Eq(carb, Carbonyl*c*(0.95-z)),                                # Change: aromatic H
    Eq(ester, Ester*c*(0.95-z)),                              # Change: aromatic H
    Eq(eth,0.5*Ether*c*(0.95-z)),                                 # Change: aromatic C-H (In multi CP-NMR ether groups count two carbons per oxygen)
    Eq(alip, Aliphatic*c*(1-h)),            
    Eq(methyl,(alip*u)/1),                                        # Change: aromatic H
    Eq(ali_chain,(alip*b)/2),                                     # Change: aromatic H
    Eq(ali_chain_2,(alip*p)/3),                                   # Change: aromatic H
    
    # Nitrogen
    Eq(nitro_groups,(MW*Nitrogen)/14.0067),
    Eq(aniline,nitro_groups*2/4),                                  # Change: aromatic H
    Eq(pyridin,nitro_groups*1/4),                                  # Change: aromatic C-H
    Eq(quaternaryN,nitro_groups*1/4),                              # Change: interior C
    
    # Hydrogen
    Eq(hydro,(h_total+methyl*2+ali_chain*4+ali_chain_2*6+nitro_groups*3/4-carb)),
    
    # Oxygen
    Eq(hydroxyl,((MW*Oxygen)/15.999)*v),                           # Change: aromatic H
    Eq(oxy,hydroxyl+ester*2+carb+eth),
    
    # Molecular weight
    Eq(MW,(c*12.011+hydro*1.00784+nitro_groups*14.0067+oxy*15.999))
     
]

# Solve the equations
solution = solve(equations, (c, carb, ester, eth, alip, c_nonar ,defe, aro_ring,methyl, ali_chain, hydro, aro, nitro_groups, aniline,pyridin, quaternaryN, MW, oxy, hydroxyl, ali_chain_2))

# Print the solutions
print("aro =", round(solution[aro]))
print("aro_ring =", round(solution[aro_ring]))
print("carb =", round(solution[carb]))
print("ester =", round(solution[ester]))
print("eth =", round(solution[eth]))
print("alip =", math.ceil(solution[alip]))
print("defe =", round(solution[defe]))
print("c_nonar =", round(solution[c_nonar]))
print("methyl =", round(solution[methyl]))
print("ali_chain =", math.ceil(solution[ali_chain]))
print("ali_chain_2 =", math.ceil(solution[ali_chain_2]))
print("aniline =", round(solution[aniline]))
print("pyridin =",  math.ceil(solution[pyridin]))
print("quaternaryN =", round(solution[quaternaryN]))
print("Hydroxyl =", round(solution[hydroxyl]))
print("MW =", round(solution[MW]))
print("Oxygen =", round(solution[carb])+round(solution[ester])*2+ round(solution[eth]) +round(solution[hydroxyl]))
print("Nitrogen =", round(solution[aniline])+math.ceil(solution[pyridin])+round(solution[quaternaryN]))
print("Carbon =", round(solution[c]))
print("Hydrogen =", round(solution[hydro]))

aro = 4817
aro_ring = 5750
carb = 151
ester = 216
eth = 400
alip = 241
defe = 859
c_nonar = 1467
methyl = 80
ali_chain = 81
ali_chain_2 = 0
aniline = 4
pyridin = 3
quaternaryN = 2
Hydroxyl = 1
MW = 106254
Oxygen = 984
Nitrogen = 9
Carbon = 7217
Hydrogen = 3662


### Create "holes" within the structure

In [29]:
if holes == 0:
    Internal_C_to_remove=0
else:
    Internal_C_to_remove = round(c_defect - solution[defe])
    print("Carbon to remove:", round(Internal_C_to_remove))

Carbon to remove: 36


In [30]:
# Adjust the excess carbon and create more realistic defects in the aromatic structures
if Internal_C_to_remove >= (num_objects*0.9):
    Internal_C_to_remove_Aro = math.ceil(Internal_C_to_remove*0.9)
else:
    Internal_C_to_remove_Aro=Internal_C_to_remove
print("Carbon to remove in aromatic structures:", Internal_C_to_remove_Aro)

Carbon to remove in aromatic structures: 36


### Define cross-links by fixing H/C ratio

In [31]:
def calculate_Err_H_C(i):
    mw = (round(solution[c])-Internal_C_to_remove)* 12.011 + i * 1.00784 + (round(solution[aniline]) + round(solution[pyridin]) + round(solution[quaternaryN]))* 14.0067 + (round(solution[carb])+round(solution[ester])*2+ round(solution[eth]) +round(solution[hydroxyl])) * 15.999
    H_C_mod = i/round(solution[c])
    Err_H_C = (abs(H_C_mod - H_C) / H_C) * 100
    return Err_H_C

# Iterate over possible values of i and find the minimum error for H/C ratio
min_error = np.inf
best_i = None
for i in range(round(system_size/2)):  
    error = calculate_Err_H_C(i)
    if error < min_error:
        min_error = error
        best_i = i

print("Hydrogen needed:", best_i)
print("Error:", round(min_error,3))

Hydrogen needed: 3695
Error: 0.003


In [32]:
Hydrogen_to_remove=round(solution[hydro])-best_i
if Hydrogen_to_remove < 0:
      Hydrogen_to_remove = 0
print("Hydrogen to remove:", Hydrogen_to_remove)
crosslinks=round(Hydrogen_to_remove/2)
print("crosslinks:", crosslinks)
newhydro=round(solution[hydro]-Hydrogen_to_remove)
newMW = (round(solution[c])-Internal_C_to_remove)* 12.011 + newhydro * 1.00784 + (round(solution[aniline]) + round(solution[pyridin]) + round(solution[quaternaryN]))* 14.0067 + (round(solution[carb])+round(solution[ester])*2+ round(solution[eth]) +round(solution[hydroxyl])) * 15.999

Hydrogen to remove: 0
crosslinks: 0


### See if you have an excess of oxygen


In [33]:
def calculate_Err_O(j):
    mw = (solution[c]-Internal_C_to_remove)* 12.011 + newhydro * 1.00784 + ((solution[aniline] +solution[pyridin] +solution[quaternaryN])* 14.0067) + j * 15.999
    OxygenMoles = (j * 15.999) / mw
    Err_O = (abs(OxygenMoles - Oxygen) / Oxygen)*100
    return Err_O

# Iterate over possible values of i and find the minimum error for O/C ratio
min_error = np.inf
best_j = None
for j in range(round(system_size/3)): 
    error = calculate_Err_O(j)
    if error < min_error:
        min_error = error
        best_j = j

print("Oxygen needed:", best_j)
print("Error:", round(min_error,3))

Oxygen needed: 992
Error: 0.018


### Before modifying check: CNMR data

In [34]:
# Aromatic Carbon
Err_C=(abs(((solution[aro]-Internal_C_to_remove_Aro)/(solution[c]-Internal_C_to_remove))-Aromatic)/Aromatic)*100
print("Aromatic Carbon:",round(((solution[aro]-Internal_C_to_remove_Aro)/(solution[c]-Internal_C_to_remove)),3))
print("Error:",round(Err_C,3))

Aromatic Carbon: 0.666
Error: 1.116


In [35]:
# Carbonyl Carbon
Err_Car=(abs((solution[carb]/(solution[c]-Internal_C_to_remove))-Carbonyl)/Carbonyl)*100
print("Carbonyl Carbon",round(solution[carb]/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_Car,3))

Carbonyl Carbon 0.021
Error: 9.549


In [36]:
# Carboxyl and Ester Carbon
Err_Carb=(abs((solution[ester]/(solution[c]-Internal_C_to_remove))-Ester)/Ester)*100
print("Ester Carbon",round(solution[ester]/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_Carb,3))

Ester Carbon 0.030
Error: 9.549


In [37]:
# Ether Carbon
Err_ethh=(abs(((solution[eth]*2)/(solution[c]-Internal_C_to_remove))-Ether)/Ether)*100
print("Ether Carbon",round((solution[eth]*2)/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_ethh,3))

Ether Carbon 0.112
Error: 9.549


In [38]:
# Aliphatic Carbon
Err_ali=(abs((solution[alip]/(solution[c]-Internal_C_to_remove))-Aliphatic)/Aliphatic)*100
print("Aliphatic Carbon",round(solution[alip]/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_ali,3))

Aliphatic Carbon 0.033
Error: 0.501


In [39]:
# Defect Carbon
Err_defe=(abs((c_defect-Internal_C_to_remove)/(solution[c]-Internal_C_to_remove)-Defect))/Defect*100
print("Defect Carbon",round((c_defect-Internal_C_to_remove)/(solution[c]-Internal_C_to_remove),3))
print("Error:",round(Err_defe,3))

Defect Carbon 0.120
Error: 5.581


### Before modifying check: Elemental composition 

In [40]:
# Carbon
Elem_c=((solution[c]-Internal_C_to_remove)*12.011)/newMW
print("Carbon:",round(Elem_c,4))
Err_elemc=(abs(Elem_c-Carbon)/Carbon)*100
print("Error:",round(Err_elemc,3))

Carbon: 0.8152
Error: 0.140


In [41]:
# Nitrogen
Elem_n=(solution[nitro_groups]*14.0067)/newMW
print("Nitrogen:",round(Elem_n,4))
Err_elemn=(abs(Elem_n-Nitrogen)/Nitrogen)*100
print("Error:",round(Err_elemn,3))

Nitrogen: 0.0011
Error: 0.432


In [42]:
# Oxygen
Elem_o=((round(solution[carb])+round(solution[ester])*2+ round(solution[eth]) +round(solution[hydroxyl]))*15.999)/newMW
print("Oxygen:",round(Elem_o,3))
Err_elemo=(abs(Elem_o-Oxygen)/Oxygen)*100
print("Error:",round(Err_elemo,3))

Oxygen: 0.149
Error: 0.665


In [43]:
# Hydrogen
Elem_h=(newhydro*1.00784)/newMW
print("Hydrogen:",round(Elem_h,3))
Err_elemh=(abs(Elem_h-Hydrogen)/Hydrogen)*100
print("Error:",round(Err_elemh,3))

Hydrogen: 0.035
Error: 0.329


## Step 4: Modify structures and include functional groups

In [44]:
object_masses= builder.Get_Object_Masses()
object_names = list(object_masses.keys())
while not object_names:
    object_masses = builder.Get_Object_Masses()
    object_names = list(object_masses.keys())

### Include Ether Groups

In [45]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_totali= atom_counts["C"]
h_totali= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7050
Element H: 3326
Element N: 0
Element O: 0
Element S: 0
Element Cl: 0
Element Si: 0


In [46]:
object_names = list(object_masses.keys())
modified_ether = []

if Biochar_Temp == 400:
    k1 = 0.85
elif Biochar_Temp == 500:
    k1 = 0.9
else:
    k1 = 1

num_molecules = round(solution[eth]*k1)

excluded_prefixes = ["circumovalene","circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84","N"]
filtered_object_names = [name for name in object_names if any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        c_aro_types = builder.examine_main(mol_to_edit)

        if len(c_aro_types['O_2n']) <= round(O_C*60):
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("carbon_not_bonded_to_oxygen", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("carbon_not_bonded_to_oxygen")
        
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value=random.choice(c_aro_types['C_2n'])
        if value in c_choices:
            break

    # Attach Ether group 
    object_ether = builder.Change_Element_CtoO(mol_to_edit,value)
    modified_ether.append(object_ether)

In [47]:
object_names = list(object_masses.keys())
modified_ether = []

if Biochar_Temp == 400:
    k = 0.15
elif Biochar_Temp == 500:
    k = 0.1
else:
    k = 0

num_molecules = round(solution[eth]*k)

excluded_prefixes = ["circumovalene","circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84","defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)


for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        c_aro_types = builder.examine_main(mol_to_edit)

        if len(c_aro_types['O_2n']) <= round(O_C*20):
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("carbon_not_bonded_to_oxygen", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("carbon_not_bonded_to_oxygen")
        
    c_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value=random.choice(c_aro_types['C_2n'])
        if value in c_choices:
            break

    # Attach Ether group 
    object_ether = builder.Change_Element_CtoO(mol_to_edit,value)
    modified_ether.append(object_ether)

In [48]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total= atom_counts["C"]
h_total= atom_counts["H"]
o_total= atom_counts["O"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6650
Element H: 2915
Element N: 0
Element O: 400
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero


In [49]:
cmd.select("carbon_bonded_to_oxygen", "elem O and neighbor elem O")

0

In [50]:
check_ether_o=o_total-round(solution[eth]*k)-round(solution[eth]*k1)
print(check_ether_o)
check_ether_c=c_totali-c_total-round(solution[eth]*k)-round(solution[eth]*k1)
print(check_ether_c)

0
0


###  Include Pyridinic Group 

In [51]:
object_names = list(object_masses.keys())
modified_pyridin = []

num_molecules = math.ceil(solution[pyridin])

excluded_prefixes = ["dibenzofuran","defect","benzo_a_fluorene", "benzo_b_fluorene","benzene","naphthalene","dibenzofuran","phenalene", "phenanthrene","anthracene","pyrene","benzo_g_h_i_perylene","benzo_a_pyrene","perylene","chrysene","defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

# Select a specific number of molecules from the filtered list
if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)


# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    
    while True:
        
        # Get available carbon types
        n_types = builder.examine_main(mol_to_edit)

        if len(n_types['N_2n'])<1:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
    
    cmd.select("nitrogen_not_bonded_to_nitrogen", f"({mol_to_edit} and elem C and neighbor (elem H) and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("nitrogen_not_bonded_to_nitrogen")
        
    n_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value_n=random.choice(n_types['C_2n'])
        if value_n in n_choices:
            break

    # Attach Pyridinic group 
    object_pyridin = cmd.alter(f'{mol_to_edit} and index {value_n}', 'elem="N"')
    object_pyridin = cmd.alter(f'{mol_to_edit} and index {value_n}', 'text_type="N.2"')
    object_pyridin = cmd.alter(f'{mol_to_edit} and index {value_n}', 'name="N"')

    # Store the modified Pyridinic
    modified_pyridin.append(object_pyridin)
    h_types = builder.examine_h(mol_to_edit)
    h_choice = random.choice(h_types['Aliphatic_N_inRing'])
    object_pyridin = cmd.remove(f'{mol_to_edit} and index {h_choice}')

    # Store the modified Pyridinic
    modified_pyridin.append(object_pyridin)

In [52]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
n_total= atom_counts["N"]
h_totali= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6647
Element H: 2912
Element N: 3
Element O: 400
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero


In [53]:
check_n=n_total-math.ceil(solution[pyridin])
print(check_n)

0


In [54]:
cmd.select("nitrogen_bonded_to_oxygen_nitrogen", "elem N and neighbor elem O")

0

In [55]:
cmd.select("nitrogen_bonded_nitrogen", "elem N and neighbor elem N")

0

### Include Hydroxyl Groups

In [56]:
object_names = list(object_masses.keys())
modified_hydroxyls = []

num_molecules = round(solution[hydroxyl])

excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    # Get available hydrogen types
    while True:
        
        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)

        if len(o_types['O_2n'])<3 and len(o_types['O_1n'])< 3:
            break  # Condition met, proceed to modify the molecule
        
        mol_to_edit = random.choice(filtered_object_names)
    
    h_types = builder.examine_h(mol_to_edit)
    
    # Check if the molecule has 'Aliphatic_C_inRing' oxygen type
    if len(h_types['Aliphatic_C_inRing']) == 0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else: 
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])

        # Attach Hydroxyl group
        object_hydroxyl = builder.Attach_OH(mol_to_edit, h_choice)
    modified_hydroxyls.append(object_hydroxyl)

In [57]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
o_total= atom_counts["O"]
c_total= atom_counts["C"]
h_total= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6647
Element H: 2912
Element N: 3
Element O: 401
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [58]:
check_ox=o_total-round(solution[eth]*k)-round(solution[eth]*k1)-round(solution[hydroxyl])
print(check_ox)
check_hy=h_totali-h_total
print(check_hy)

0
0


### Include Carbonyl Groups

In [59]:
object_names = list(object_masses.keys())
modified_carbonyl = []

num_molecules = round(solution[carb])

excluded_prefixes = ["benzo_a_fluorene", "benzo_b_fluorene","phenalene", "phenanthrene","anthracene","pyrene","benzo_a_pyrene","perylene","chrysene","benzo_b_fluoranthene","N5"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)


for mol_to_edit in molecules_to_edit:
    while True:

        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")
            
        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]
        
        if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*40) and len(o_types['O_1n'])<round(O_C*40):
            break  # Condition met, proceed to modify the molecule
            
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit) 

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Carbonyl group
    object_carbonyl = builder.Attach_Carbonyl(mol_to_edit, h_choice)
    modified_carbonyl.append(object_carbonyl)

In [60]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
o_total_new= atom_counts["O"]
c_total_new= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 6798
Element H: 2912
Element N: 3
Element O: 552
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [61]:
check_c=c_total_new-(c_total+round(solution[carb]))
print(check_c)
check_o=o_total_new-(o_total+round(solution[carb]))
print(check_o)

0
0


### Include Ester and Carboxyl Groups

In [62]:
object_names = list(object_masses.keys())
modified_ester = []

num_molecules = math.floor(solution[ester]*0.9) # 90% as ester groups

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    while True:
            
        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]
        
        if Biochar_Temp==700:
            if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*80) and len(o_types['O_1n'])<round(O_C*80):
                break  # Condition met, proceed to modify the molecule
        else:
             if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*55) and len(o_types['O_1n'])<round(O_C*55):
                break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing']) == 0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Ester group
    object_ester = builder.Attach_Ester(mol_to_edit, h_choice)
    modified_ester.append(object_ester)

In [63]:
object_names = list(object_masses.keys())
modified_carbox = []

num_molecules = math.ceil(solution[ester]*0.1) # 10% as carboxylic groups

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    while True:
            
        # Get available carbon types
        o_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]
        
        if Biochar_Temp==700:
            if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*80) and len(o_types['O_1n'])<round(O_C*80):
                break  # Condition met, proceed to modify the molecule
        else:
             if len(h_choices)>3 and len(o_types['O_2n'])<round(O_C*55) and len(o_types['O_1n'])<round(O_C*55):
                break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing']) == 0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Carboxylic group
    object_carbox = builder.Attach_Carbox(mol_to_edit, h_choice)
    modified_carbox.append(object_carbox)

In [64]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
o_total_new2= atom_counts["O"]
c_total_new2= atom_counts["C"]
h_total_new2= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7014
Element H: 2912
Element N: 3
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [65]:
check_c=c_total_new2-(c_total_new+round(solution[ester]))
print(check_c)
check_o=o_total_new2-(o_total_new+round(solution[ester])*2)
print(check_o)

0
0


### Include Methyl Groups

In [66]:
object_names = list(object_masses.keys())
modified_methyl = []

num_molecules = round(solution[methyl])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    while True:  

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>4:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break
            
    # Attach Methyl group 
    object_methyl = builder.Attach_CH3(mol_to_edit, h_choice)
    modified_methyl.append(object_methyl)

In [67]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total_new= atom_counts["C"]
h_total_new= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7094
Element H: 3072
Element N: 3
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [68]:
check_c=c_total_new-(c_total_new2+round(solution[methyl]))
print(check_c)
check_hy=h_total_new-(h_total_new2+round(solution[methyl])*2)
print(check_hy)

0
0


### Include CH2CH3 Groups

In [69]:
object_names = list(object_masses.keys())
modified_ali_chain = []

num_molecules = round(solution[ali_chain])

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>8:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Aliphatic group           
    object_ali_chain = builder.Attach_CH2CH3(mol_to_edit, h_choice)
    modified_ali_chain.append(object_ali_chain)

In [70]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total= atom_counts["C"]
h_total= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7254
Element H: 3392
Element N: 3
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [71]:
check_c=c_total-(c_total_new+round(solution[ali_chain])*2)
print(check_c)
check_hy=h_total-(h_total_new+round(solution[ali_chain])*4)
print(check_hy)

0
0


### Include (CH2)2-CH3 Groups for Biochar at 400°C

In [72]:
object_names = list(object_masses.keys())
modified_ali_chain_2 = []

num_molecules = math.ceil(solution[ali_chain_2]/2)

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>10:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    if len(h_types['Aliphatic_C_inRing'])==0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else:
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])
     
    # Attach Aliphatic chain group           
    object_ali_chain2 = builder.Attach_C2H4CH3(mol_to_edit, h_choice)
    modified_ali_chain_2.append(object_ali_chain2)

In [73]:
object_names = list(object_masses.keys())
modified_ali_chain_2 = []

num_molecules = math.ceil(solution[ali_chain_2]/2)

if num_molecules > len(object_names):
    molecules_to_edit = random.choices(object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(object_names, num_molecules)

for mol_to_edit in molecules_to_edit:     
    while True:

        # Get available hydrogen types
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(h_choices)>12:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
        
    h_types = builder.examine_h(mol_to_edit)

    if len(h_types['Aliphatic_C_inRing'])==0:
        h_choice = random.choice(h_types['Aliphatic_C'])
    else:
        h_choice = random.choice(h_types['Aliphatic_C_inRing'])
     
    # Attach Aliphatic chain group           
    object_ali_chain2 = builder.Attach_C2H4CH3(mol_to_edit, h_choice)
    modified_ali_chain_2.append(object_ali_chain2)

In [74]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
c_total_2= atom_counts["C"]
h_total_2= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7254
Element H: 3392
Element N: 3
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [75]:
check_c=c_total_2-(c_total+math.ceil(solution[ali_chain_2])*3)
print(check_c)
check_hy=h_total_2-(h_total+math.ceil(solution[ali_chain_2])*6)
print(check_hy)

0
0


### Include NH3 Groups

In [76]:
object_names = list(object_masses.keys())
modified_Aniline = []

num_molecules = round(solution[aniline])

excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:           
    while True:

        # Get available carbon types
        n_types = builder.examine_main(mol_to_edit)
        cmd.select("hydro_not_bonded_to_CO", f"({mol_to_edit} and elem H and neighbor (elem C) and (not neighbor (elem O)))")

        # Get the selection of carbon atoms not bonded to oxygen
        selection_dict = cmd.get_model("hydro_not_bonded_to_CO")
        h_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("H")]

        if len(n_types['N_2n'])==0 and len(h_choices)>7 and len(n_types['N_3n']) == 0 and len(n_types['N_1n']) == 0:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)

    h_types = builder.examine_h(mol_to_edit)

    while True:
        if len(h_types['Aliphatic_C_inRing'])==0:
            h_choice = random.choice(h_types['Aliphatic_C'])
        else:
            h_choice = random.choice(h_types['Aliphatic_C_inRing'])
        if h_choice in h_choices:
            break

    # Attach Aniline group
    object_Aniline = builder.Attach_Aniline(mol_to_edit, h_choice)
    modified_Aniline.append(object_Aniline)

In [77]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
n_total= atom_counts["N"]
h_total_new= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7254
Element H: 3396
Element N: 7
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero

In [78]:
check_n=n_total-(round(solution[aniline])+ math.ceil(solution[pyridin]))
print(check_n)
check_h=h_total_new-(h_total_2+round(solution[aniline]))
print(check_h)

0
0


### Include Quaternary Nitrogen

In [79]:
object_names = list(object_masses.keys())
modified_quater = []

num_molecules = round(solution[quaternaryN])

excluded_prefixes = ["circumcoronene","circumpyrene","pentatriacotaene", "circumcircumpyrene","C84"]
filtered_object_names = [name for name in object_names if any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    while True:
        # Get available carbon types
        n_types = builder.examine_main(mol_to_edit)

        if len(n_types['N_2n']) == 0 and len(n_types['N_3n']) == 0 and len(n_types['N_1n']) == 0:
            break  # Condition met, proceed to modify the molecule
        
        # Select a new molecule from the list
        mol_to_edit = random.choice(filtered_object_names)
            
    cmd.select("nitrogen_not_bonded_to_nitrogen", f"({mol_to_edit} and elem C and not neighbor (elem O))")

    # Get the selection of carbon atoms not bonded to oxygen
    selection_dict = cmd.get_model("nitrogen_not_bonded_to_nitrogen")
        
    n_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value_n=random.choice(n_types['C_3n'])
        if value_n in n_choices:
            break
    # Change the Carbon selected to Nitrogen
    object_quater = builder.Change_Element_CtoN(mol_to_edit,value_n)
    modified_quater.append(object_quater)

In [80]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
n_total_neww= atom_counts["N"]
c_total_neww= atom_counts["C"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7252
Element H: 3396
Element N: 9
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check these values: Should be zero


In [81]:
check_cc=c_total_2-(c_total_neww+round(solution[quaternaryN]))
print(check_cc)
check_n=n_total_neww-(n_total+round(solution[quaternaryN]))
print(check_n)

0
0


### Correct atom valence and include hydrogens to fix the inclusion of functional groups

In [82]:
cmd.do("valence guess, all")

In [83]:
def add_hydrogens_excluding_prefixes():
    excluded_prefixes = ["defect"]
    
    filtered_object_names = [
        name for name in object_names
        if not any(name.startswith(prefix) for prefix in excluded_prefixes)
    ]
    
    for name in filtered_object_names:
        cmd.h_add(name)
        
add_hydrogens_excluding_prefixes()

In [84]:
cmd.do('colour')
cmd.do('orient')
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')
cmd.bg_color("white")
cmd.set("ray_shadows", "off")

In [85]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalC1= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]
FinalN= atom_counts["N"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7252
Element H: 3761
Element N: 9
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


In [86]:
cmd.do("valence guess, all")

In [87]:
object_names = list(object_masses.keys())
for obj in object_names:
    cmd.do('clean %s' % obj)
    cmd.do('scultp %s' % obj)

In [88]:
builder.clear_label()

## Step 5: Create a cross-linked network to fix H/C ratio

### Obtain the molecular formula of decorated clusters before crosslinking

In [89]:
object_list = cmd.get_object_list()
def get_atom_counts(object_list):
    elements = ['C', 'H', 'N', 'O', 'S', 'Cl', 'Si']
    counts = [object_list]  # Start with the molecule name
    for element in elements:
        count = cmd.count_atoms(f'elem {element} and model {object_list}')
        counts.append(count)
    return counts

with open('Molecular_formula_before_crosslinking.txt', 'w') as file:
    file.write("Molecule,C,H,N,O,S,Cl,Si\n")
    
    for molecule in object_list:
        atom_counts = get_atom_counts(molecule)
        file.write(",".join(map(str, atom_counts)) + '\n')

print("Composition analysis complete. Results saved to 'Molecular_formula_before_crosslinking.txt'.")

Composition analysis complete. Results saved to 'Molecular_formula_before_crosslinking.txt'.


### Check againg cross-links by fixing H/C ratio

In [90]:
def calculate_Err_H_C(i):
    mw = (FinalC1-Internal_C_to_remove)* 12.011 + i * 1.00784 + FinalN* 14.0067 + FinalO * 15.999
    H_C_mod = i/(FinalC1-Internal_C_to_remove)
    Err_H_C = (abs(H_C_mod - H_C) / H_C) * 100
    return Err_H_C

# Iterate over possible values of i and find the minimum error for H/C ratio
min_error = np.inf
best_i = None
for i in range(6000):  # Adjust the range as needed based on how many carbons you have in the simulation
    error = calculate_Err_H_C(i)
    if error < min_error:
        min_error = error
        best_i = i

print("Best i value:", best_i)
print("Minimum Err_H_C:", round(min_error,3))

Best i value: 3695
Minimum Err_H_C: 0.011


#### The number of cross-links needs to be an even number

In [91]:
def round_to_lowest_even(number):
    divided_number = number / 2
    rounded_number = math.floor(divided_number)
    lowest_even_number = rounded_number * 2
    return lowest_even_number

In [92]:
Hydrogen_to_remove=round_to_lowest_even(FinalH-best_i)
print("Hydrogen to be removed:",Hydrogen_to_remove)
crosslinks=round_to_lowest_even(Hydrogen_to_remove/2)
print("Cross-links needed:",crosslinks)
newhydro=FinalH-Hydrogen_to_remove
newMW=(FinalC1-Internal_C_to_remove)* 12.011 + newhydro * 1.00784 + FinalN* 14.0067 + FinalO * 15.999

Hydrogen to be removed: 66
Cross-links needed: 32


In [93]:
object_list = cmd.get_object_list()
num_objects = len(object_list)
if crosslinks >= (num_objects*0.75):
    crosslinks=round_to_lowest_even(crosslinks*0.65)
elif crosslinks <=0 or crosslinks < 20:
    crosslinks=round_to_lowest_even(FinalH*0.01)
else:
    crosslinks=crosslinks
print("Cross-links needed:",crosslinks)
newhydro=FinalH-crosslinks*2

Cross-links needed: 32


In [94]:
HydrogenMoles = (newhydro * 1.00784) / newMW
CarbonMoles = ((FinalC1-Internal_C_to_remove) * 12.011) / newMW
H_C_mod = (HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("H/C ratio:",round(H_C_mod,3))
Bridgehead_mod =1- (HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("Bridgehead Carbon:",round(Bridgehead_mod,3))

H/C ratio: 0.512
Bridgehead Carbon: 0.488


### This step is divide 6 times to create a diverse distribution

In [95]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

### Step one: 25% of the total cross-links

In [96]:
excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names, round_to_lowest_even(crosslinks/2))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) >1 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

16


In [97]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [98]:
modified_cross = [None] * round_to_lowest_even(crosslinks/4)
mol_to_edit = [None] * round_to_lowest_even(crosslinks/2)
h_types = [None] * round_to_lowest_even(crosslinks/2)
h_choice = [None] *round_to_lowest_even(crosslinks/2)

for i in range(0,  round_to_lowest_even(crosslinks/4)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/4), round_to_lowest_even(crosslinks/2)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [99]:
for k in range(0, round_to_lowest_even(crosslinks/4)):
    modified_cross[k]= builder.Crosslink_mols1(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/4)], h_choice[k+round_to_lowest_even(crosslinks/4)])

In [100]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalN= atom_counts["N"]
FinalC_1= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7252
Element H: 3745
Element N: 9
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero


In [101]:
Totalc=FinalC_1-FinalC1
print(Totalc)

0


### Step two: + 25% of the total cross-links

In [102]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [103]:
excluded_prefixes = ["defect"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names, round_to_lowest_even(crosslinks/2))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) >1 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

16


In [104]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [105]:
modified_cross = [None] * round_to_lowest_even(crosslinks/4)
mol_to_edit = [None] * round_to_lowest_even(crosslinks/2)
h_types = [None] * round_to_lowest_even(crosslinks/2)
h_choice = [None] *round_to_lowest_even(crosslinks/2)

for i in range(0,  round_to_lowest_even(crosslinks/4)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/4), round_to_lowest_even(crosslinks/2)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [106]:
for k in range(0, round_to_lowest_even(crosslinks/4)):
    modified_cross[k]= builder.Crosslink_mols2(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/4)], h_choice[k+round_to_lowest_even(crosslinks/4)])

In [107]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalN= atom_counts["N"]
FinalC2= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7252
Element H: 3729
Element N: 9
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [108]:
Totalc=FinalC2-FinalC1
print(Totalc)

0


### Step three: + 12.5% of the total cross-links

In [109]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [110]:
object_names_new = list(object_masses.keys())
excluded_prefixes = ["defect","combined"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]
molecules_to_edit = random.sample(filtered_object_names,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) >2 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

8


In [111]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [112]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [113]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols3(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

In [114]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalN= atom_counts["N"]
FinalC3= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7252
Element H: 3721
Element N: 9
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [115]:
Totalc2=FinalC3-FinalC1
print(Totalc2)

0


### Step four: + 12.5% of the total cross-links

In [116]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [117]:
print(len(list(object_masses.keys())))

100


In [118]:
object_names_new = list(object_masses.keys())
molecules_to_edit = random.sample(object_names_new,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) > 2 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

8


In [119]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [120]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [121]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols4(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

In [122]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalN= atom_counts["N"]
FinalC4= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7252
Element H: 3713
Element N: 9
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [123]:
Totalc3=FinalC4-FinalC1
print(Totalc3)

0


### Step five: + 12.5% of the total cross-links

In [124]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [125]:
print(len(list(object_masses.keys())))

96


In [126]:
object_names_new = list(object_masses.keys())
molecules_to_edit = random.sample(object_names_new,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) > 2 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

8


In [127]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [128]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [129]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols5(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

In [130]:
# Count the number of atoms for each element
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
FinalN= atom_counts["N"]
FinalC5= atom_counts["C"]
FinalO= atom_counts["O"]
FinalH= atom_counts["H"]
# Print the atom counts
for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7252
Element H: 3705
Element N: 9
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [131]:
Totalc4=FinalC5-FinalC1
print(Totalc4)

0


### Step six: final 12.5% of the total cross-links

In [132]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [133]:
print(len(list(object_masses.keys())))

92


In [134]:
object_names_new = list(object_masses.keys())
molecules_to_edit = random.sample(object_names_new,round_to_lowest_even(crosslinks/4))

selected_objects = []

# Loop over each selected molecule
for mol_to_edit in molecules_to_edit:
    while True:
        # Get available carbon types
        h_types = builder.examine_h(mol_to_edit)

        if len(h_types['Aliphatic_C_inRing']) > 2 and mol_to_edit not in selected_objects:
            selected_objects.append(mol_to_edit)  # Condition met, append to selected_objects
            break  # Condition met, proceed to modify the molecule
        else:
            # Select a new molecule from molecules_to_edit
            mol_to_edit = random.choice(filtered_object_names)

print(len(selected_objects))

8


In [135]:
# Check for repeated elements
has_duplicates = len(selected_objects) != len(set(selected_objects))

if has_duplicates:
    print("There are repeated elements in the list.")
else:
    print("There are no repeated elements in the list.")

There are no repeated elements in the list.


In [136]:
mol_to_edit = [None] * round_to_lowest_even(crosslinks/4)
h_types = [None] * round_to_lowest_even(crosslinks/4)
h_choice = [None] *round_to_lowest_even(crosslinks/4)
for i in range(0, round_to_lowest_even(crosslinks/8)):
    mol_to_edit[i]= selected_objects[i]
    h_types[i] = builder.examine_h(mol_to_edit[i])
    h_choice[i] = random.choice(h_types[i]['Aliphatic_C_inRing'])
for j in range(round_to_lowest_even(crosslinks/8), round_to_lowest_even(crosslinks/4)):
    mol_to_edit[j] = selected_objects[j]
    h_types[j] = builder.examine_h(mol_to_edit[j])
    h_choice[j] = random.choice(h_types[j]['Aliphatic_C_inRing'])

In [137]:
for k in range(0, round_to_lowest_even(crosslinks/8)):
    modified_cross[k]= builder.Crosslink_mols6(mol_to_edit[k], h_choice[k], mol_to_edit[k+round_to_lowest_even(crosslinks/8)], h_choice[k+round_to_lowest_even(crosslinks/8)])

### Create holes: Aromatic structures

In [138]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [139]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
Final_N= atom_counts["N"]
Final_C1= atom_counts["C"]
Final_O= atom_counts["O"]
Final_H= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7252
Element H: 3697
Element N: 9
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


In [140]:
modified_hole = []
num_molecules = Internal_C_to_remove

excluded_prefixes = ["defect","fluorine","phenalene", "phenanthrene","anthracene","tetracene","pentacene","pyrene","chrysene","benzo_a_fluorene","benzo_b_fluoranthene","benzo_b_fluorene","coronene","benzo_a_pyrene","benzo_g_h_i_perylene","m","N"]
filtered_object_names = [name for name in object_names_new if not any(name.startswith(prefix) for prefix in excluded_prefixes)]

if num_molecules > len(filtered_object_names):
    molecules_to_edit = random.choices(filtered_object_names, k=num_molecules)
else:
    molecules_to_edit = random.sample(filtered_object_names, num_molecules)

for mol_to_edit in molecules_to_edit:
    
    c_types = builder.examine_main(mol_to_edit)
    cmd.select("C-C-C", f"({mol_to_edit} and elem C and not neighbor (elem H) and not neighbor (elem O) and not neighbor ((elem C) and neighbor (elem H)))")
    
    # Get the selection of atoms
    selection_dict = cmd.get_model("C-C-C")
    c_3n_choices = [atom["index"] for atom in selection_dict["atom"] if atom["name"].startswith("C")]
    
    while True:
        value_c = random.choice(c_types['C_3n'])
        if value_c in c_3n_choices:
            break
   
    # Remove the internal carbon atom to create a hole in the structure
    object_hole = cmd.remove(f'{mol_to_edit} and index {value_c}')
    modified_hole.append(object_hole)

In [174]:
atom_counts = {}
elements = ["C", "H", "N", "O", "S", "Cl", "Si"]

for element in elements:
    count = cmd.select(f"elem {element}")
    atom_counts[element] = count
Final_N= atom_counts["N"]
Final_C= atom_counts["C"]
Final_O= atom_counts["O"]
Final_H= atom_counts["H"]

for element, count in atom_counts.items():
    print(f"Element {element}: {count}")

Element C: 7216
Element H: 3700
Element N: 9
Element O: 984
Element S: 0
Element Cl: 0
Element Si: 0


#### Check this value: Should be zero

In [175]:
check_c=Final_C1-(Final_C+Internal_C_to_remove_Aro)
print(check_c)

0


In [176]:
cmd.do("valence guess, all")

## Step 6: Check the final ultimate analysis

In [177]:
builder.clear_label()

In [178]:
newMW2=Final_C* 12.011 + Final_H * 1.00784 + Final_N* 14.0067 + Final_O * 15.999
print("MW:",round(newMW2))

MW: 106269


### Ultimate Analysis

In [179]:
# Carbon
Elem_c=(Final_C*12.011)/newMW2
print("Total C:",round(Elem_c,4))
Err_elemc=(abs(Elem_c-Carbon)/Carbon)*100
print("Final Error:",round(Err_elemc,3))

Total C: 0.8156
Final Error: 0.182


In [180]:
# Nitrogen
Elem_n=(Final_N*14.0067)/newMW2
print("Total N:",round(Elem_n,5))
Err_elemn=(abs(Elem_n-Nitrogen)/Nitrogen)*100
print("Final Error:",round(Err_elemn,3))

Total N: 0.00119
Final Error: 7.839


In [181]:
# Oxygen
Elem_o=(Final_O*15.999)/newMW2
print("Total O:",round(Elem_o,3))
Err_elemo=(abs(Elem_o-Oxygen)/Oxygen)*100
print("Final Error:",round(Err_elemo,3))

Total O: 0.148
Final Error: 1.107


In [182]:
# Hydrogen
Elem_h=Final_H*1.00784/newMW2
print("Total H:",round(Elem_h,3))
Err_elemh=(abs(Elem_h-Hydrogen)/Hydrogen)*100
print("Final Error:",round(Err_elemh,3))

Total H: 0.035
Final Error: 0.257


In [183]:
# H/C ratio
CarbonMoles = (Final_C * 12.011) / newMW2
HydrogenMoles = (Final_H * 1.00784) / newMW2
H_C_mod =(HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("H/C Ratio:",round(H_C_mod,4))
Err_H_C = (abs(H_C_mod - H_C) / H_C) * 100
print("Error:",round(Err_H_C,3))

H/C Ratio: 0.5127
Error: 0.146


In [184]:
# O/C ratio
OxygenMoles = (Final_O * 15.999) / newMW2
O_C_mod =1/((CarbonMoles * 15.999)/(OxygenMoles  * 12.011)) 
print("O/C Ratio:",round(O_C_mod,4))
Err_C_O = (abs(O_C_mod- O_C) / O_C) * 100
print("Error:",round(Err_C_O,3))

O/C Ratio: 0.1364
Error: 0.464


In [185]:
# Bridgehead carbon
brigd =1-(HydrogenMoles * 12.011) / (CarbonMoles * 1.00784)
print("Bridgehead Carbon:",round(brigd,3))
Err_brigd = (abs(brigd - BridgheadCarbon) / BridgheadCarbon) * 100
print("Error:",round(Err_brigd,3))

Bridgehead Carbon: 0.487
Error: 0.154


In [186]:
cmd.save("biochar.pse")

## Step 7: Check the final 13C NMR 

### CNMR

In [187]:
# Aromatic Carbon
Err_C=(abs(((solution[aro]-Internal_C_to_remove_Aro)/Final_C)-Aromatic)/Aromatic)*100
print("Aromatic Carbon:",round(((solution[aro]-Internal_C_to_remove_Aro)/(Final_C)),3))
print("Error:",round(Err_C,3))

Aromatic Carbon: 0.663
Error: 1.597


In [188]:
# Carbonyl Carbon
Err_Carb=(abs(((solution[carb])/Final_C)-Carbonyl)/Carbonyl)*100
print("Carbonyl Carbon:",round(((solution[carb])/(Final_C)),3))
print("Error:",round(Err_Carb,3))

Carbonyl Carbon: 0.021
Error: 9.989


In [189]:
# Carboxyl/Lactone/Ester Carbon
Err_Carbox=(abs(((solution[ester])/Final_C)-Ester)/Ester)*100
print("Ester Carbon:",round(((solution[ester])/(Final_C)),3))
print("Error:",round(Err_Carbox,3))

Ester Carbon: 0.030
Error: 9.989


In [190]:
# Ether Carbon
Err_ethh=(abs(((solution[eth]*2)/(Final_C))-Ether)/Ether)*100
print("Ether Carbon",round((solution[eth]*2)/(Final_C),3))
print("Error:",round(Err_ethh,3))

Ether Carbon 0.111
Error: 9.989


In [191]:
# Aliphatic Carbon
Err_ali=(abs(((solution[alip])/(Final_C))-Aliphatic)/Aliphatic)*100
print("Aliphatic Carbon",round((solution[alip])/(Final_C),3))
print("Error:",round(Err_ali,3))

Aliphatic Carbon 0.033
Error: 0.013


In [192]:
# Defective Carbon
Err_def=(abs(((solution[defe])/(Final_C))-Defect)/Defect)*100
print("Defective Carbon",round((solution[defe])/(Final_C),3))
print("Error:",round(Err_def,3))

Defective Carbon 0.119
Error: 5.013


### Contribution of various C-X bonds for Aromatic and defective rings carbons (Theoretical value estimated through DFT)

### C-H 

In [193]:
# Function to count bonds between specified atoms
def count_bonds(atom1, atom2):
    selection_name = f"C-H"
    cmd.select(selection_name, f"({atom1}) within 1.8 of ({atom2})")
    bond_count = cmd.count_atoms(selection_name) - 1  # Subtract 1 to exclude the central atom
    return bond_count
    
carbon_atom = "elem C"
hydrogen_atom = "elem H"

carbon_hydrogen_bonds = count_bonds(carbon_atom, hydrogen_atom)
print(f"Number of C-H bonds: {carbon_hydrogen_bonds}")

Number of C-H bonds: 2846


In [194]:
# Take into account just bonds related to aromatic and defective carbons
print("C-H:",round((carbon_hydrogen_bonds-solution[methyl]*3-solution[ali_chain]*5-solution[ali_chain_2]*5-solution[carb]-solution[eth])/(solution[aro]+solution[defe]),3))
Err_CH = (abs((carbon_hydrogen_bonds-solution[methyl]*3-solution[ali_chain]*5-solution[ali_chain_2]*5-solution[carb]-solution[eth])/(solution[aro]+solution[defe])- C_H_)/C_H_) * 100
print("Error:",round(Err_CH,3))

C-H: 0.291
Error: 6.027


### C-O-H

In [195]:
def count_three_atom_bonds(atom1, atom2, atom3):
    selection_name = f"C-O-H"
    cmd.select(selection_name, f"(({atom1}) within 1.8 of ({atom2})) or (({atom2}) within 1.8 of ({atom3})) or (({atom1}) within 1.8 of ({atom3}))")
    bond_count = cmd.count_atoms(selection_name) - 2  # Subtract 2 to exclude the central atoms
    
    return bond_count

carbon_atom = "elem C"
oxygen_atom = "elem O"
hydrogen_atom = "elem H"

carbon_oxygen_hydrogen_bonds = count_three_atom_bonds(carbon_atom, oxygen_atom, hydrogen_atom)
print(f"Number of C–O–H bonds: {carbon_oxygen_hydrogen_bonds}")

Number of C–O–H bonds: 3494


In [196]:
# Take into account just bonds related to aromatic and defective carbons
print("C-x-H:",round((carbon_oxygen_hydrogen_bonds-solution[carb]-solution[ester]*2-solution[eth])/(solution[aro]+solution[defe]),3))
Err_CHx = (abs((carbon_oxygen_hydrogen_bonds-solution[carb]-solution[ester]*2-solution[eth])/(solution[aro]+solution[defe])- C_x_H_)/C_x_H_) * 100
print("Error:",round(Err_CHx,3))

C-x-H: 0.442
Error: 5.917


## Step 8: Organize the grid again and proceed to fix the helium density in LAMMPS

In [197]:
object_masses= builder.Get_Object_Masses()
object_names_new = list(object_masses.keys())
while not object_names_new:
    object_masses = builder.Get_Object_Masses()
    object_names_new = list(object_masses.keys())

In [198]:
min_angle = 1
max_angle = 359.0

num_objects = len(object_names_new)

positions = []
used_positions = set()
angles = []
xmin, xmax = 0, round(system_size*0.01)
ymin, ymax = -round(system_size*0.01), 0
zmin, zmax = 0, round(system_size*0.01)

# Minimum distance between any two positions to avoid overlap
min_distance = round(system_size*0.001)
grid_divisions = int(math.ceil(num_objects ** (1 / 3)))

grid_size_x = min_distance
grid_size_y = min_distance
grid_size_z = min_distance

# Generate all possible grid positions
all_positions = [
    (xmin + i * grid_size_x, ymin + j * grid_size_y, zmin + k * grid_size_z)
    for i in range(grid_divisions)
    for j in range(grid_divisions)
    for k in range(grid_divisions)
]

random.shuffle(all_positions)
assert len(all_positions) >= num_objects, "Not enough positions for all objects."

# Assign positions to objects
for idx, name in enumerate(object_names_new):
    x, y, z = all_positions[idx]
    positions.append((x, y, z))

    rotation_angles = [random.uniform(min_angle, max_angle) for _ in range(3)]
    angles.append((rotation_angles, name))

    for axis, angle in zip(['x', 'y', 'z'], rotation_angles):
        rotation_command = "rotate {0}, {1}, {2}".format(axis, angle, name)
        cmd.do(rotation_command)

    # Translate the object to the new position
    cmd.translate((x, y, z), name)

cmd.refresh()

object_list = cmd.get_object_list()
num_objects = len(object_list)
print("Number of objects:", num_objects)

Number of objects: 88


In [199]:
# Obtain molecular weight distribution
object_masses = builder.Get_Object_Masses()
while not object_masses:
    object_masses = builder.Get_Object_Masses()

### Obtain molecular formula of the decorated clusters

In [200]:
def get_atom_counts(object_names_new):
    elements = ['C', 'H', 'N', 'O', 'S', 'Cl', 'Si']
    counts = [object_names_new]  # Start with the molecule name
    for element in elements:
        count = cmd.count_atoms(f'elem {element} and model {object_names_new}')
        counts.append(count)
    return counts

with open('Final_molecular_formula.txt', 'w') as file:
    file.write("Molecule,C,H,N,O,S,Cl,Si\n")
    
    for molecule in object_names_new:
        atom_counts = get_atom_counts(molecule)
        file.write(",".join(map(str, atom_counts)) + '\n')

print("Composition analysis complete. Results saved to 'Final_molecular_formula.txt'.")

Composition analysis complete. Results saved to 'Final_molecular_formula.txt'.


### Export the molecules as PDB file

In [201]:
# Select all atoms
cmd.select('all_atoms', 'all')

# Export the selection as a multi-file PDB
n_states = cmd.count_states('all_atoms')
pdb_files = []
for state in range(1, n_states + 1):
    cmd.frame(state)
    pdb_file = f'exported_molecule_{state}.pdb'
    cmd.save(pdb_file, 'all_atoms')
    pdb_files.append(pdb_file)

# Merge the multi-file PDB into a single file
merged_file = 'merged_molecule.pdb'
with open(merged_file, 'w') as outfile:
    for pdb_file in pdb_files:
        with open(pdb_file, 'r') as infile:
            outfile.write(infile.read())

# Save the merged molecule
cmd.load(merged_file, 'merged_molecule')
cmd.save('final_molecule.pdb', 'merged_molecule')

# Clean up the intermediate files
for pdb_file in pdb_files:
    os.remove(pdb_file)
os.remove(merged_file)

In [202]:
# Call the PyMOL aliases
cmd.do('colour')
cmd.do('orient')
# Change colors ans appereance
cmd.do('set sphere_scale, 0.2, (all)')
cmd.do('set_bond stick_radius, 0.14, (all), (all)')
cmd.do('show sticks')
cmd.do('show spheres')

## Step 9: Convert the PDB file to Lammps data using OVITO

#### To define the x, y, z values use the following code:

In [203]:
HeliumDensity = float(input("Enter the value for Helium Density: "))  # kg/m3

Enter the value for Helium Density:  1.470


In [171]:
Temperature=298                             # K
Pressure=1                                  # atm
Density=(HeliumDensity*1000)/(10**(27))     # g/Å3
NA=6.022*10**(23)                           # molecules per mol
BoxVolume=1/((Density/newMW2)*NA)           # Å3
BoxLength=(BoxVolume)**(1/3)                # Å
print(" x, y, z:",round(BoxLength,3))


 x, y, z: 49.33


#### Open the PDB file in VMD, and in the console put the following commands:

cd Desktop

pbc set {x y z} -all


pbc box -center all

topo writelammpsdata biochar.data charge

You can also convert it directly using OVITO, just open the file and export it as LAMMPS data using charge